In [136]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
from datetime import datetime, timedelta
import openpyxl 
import os
from pathlib import Path
import xlsxwriter
import send_email

pd.set_option('display.max_columns', None)


In [137]:
#Publisher Config
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
publisher_config = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
pub_config = publisher_config[0].get_as_df()
pub_config.rename(columns={'DP.DS or DP.sV': 'DP.SV'}, inplace=True)

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


In [138]:
#AR Smartsheet
sms_AR = infrastructure.get_smartsheet('ar_sms')


In [139]:
sms_AR['Offer ID'] = sms_AR['New Offer Name'].str.split('_').str[-1]


In [140]:
#manual input
df = pd.read_csv('SS_LC_merged_data.csv')

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [141]:
# Filter the DataFrame by Send Strategy
ar_df = df[df['Send Strategy'] == 'AR']


In [142]:
ar_df['shortcode_DP.SV'] = ar_df['shortcode_DP.SV'].apply(lambda x: str(x)[:min(3, str(x).index('_'))] + str(x)[str(x).index('_'):] if '_' in str(x) else str(x))

<ipython-input-142-2f2f4f8aaa10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar_df['shortcode_DP.SV'] = ar_df['shortcode_DP.SV'].apply(lambda x: str(x)[:min(3, str(x).index('_'))] + str(x)[str(x).index('_'):] if '_' in str(x) else str(x))


In [143]:
# remove only the ".0" at the end of the Hitpath_Offer_ID column in ar_df
ar_df['Hitpath_Offer_ID'] = ar_df['Hitpath_Offer_ID'].str.replace(r'\.0$', '')


<ipython-input-143-c0d54c8234f7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  ar_df['Hitpath_Offer_ID'] = ar_df['Hitpath_Offer_ID'].str.replace(r'\.0$', '')
<ipython-input-143-c0d54c8234f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar_df['Hitpath_Offer_ID'] = ar_df['Hitpath_Offer_ID'].str.replace(r'\.0$', '')


Last 30 day data

In [144]:

# Convert 'Date' column to datetime type
ar_df['Date'] = pd.to_datetime(ar_df['Date'])

# Calculate the date 37 days ago
thirty_days_ago = datetime.now() - timedelta(days=37)

# Calculate the date 7 days ago
seven_days_ago = datetime.now() - timedelta(days=7)

# Filter the DataFrame for the last 30 days after the last 7 days
ar30_df = ar_df[(ar_df['Date'] <= seven_days_ago) & (ar_df['Date'] >= thirty_days_ago)]

# Filter the DataFrame for the last 7 days
ar7_df = ar_df[ar_df['Date'] >= seven_days_ago]

<ipython-input-144-fac7a05c68f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar_df['Date'] = pd.to_datetime(ar_df['Date'])


In [145]:
#filter 30 day df
filtered_ar30_df = ar30_df[['Hitpath_Offer_ID', 'Affiliate_Id', 'DP&Pub', 'Revenue', 
                            'Jump Page Clicks', 'Delivered', 'Optout', 'Clicks', 'Cost', 
                            'Ar Day', 'Shortcode Name', 'DP.SV', 'shortcode_DP.SV', 
                            'Opportunity Cost', 'Offer Vertical']]

#filter 7 day df
filtered_ar7_df = ar7_df[['Hitpath_Offer_ID', 'Affiliate_Id', 'DP&Pub', 'Revenue',
                            'Jump Page Clicks', 'Delivered', 'Optout', 'Clicks', 'Cost',
                            'Ar Day', 'Shortcode Name', 'DP.SV', 'shortcode_DP.SV',
                            'Opportunity Cost', 'Offer Vertical']]

In [146]:
grouped_ar30_df = filtered_ar30_df.groupby(['shortcode_DP.SV', 'Ar Day']).agg({'Revenue': 'sum', 'Cost': 'sum', 'Delivered': 'sum', 'Jump Page Clicks': 'sum', 'Clicks': 'sum', 'Opportunity Cost': 'sum', 'Optout': 'sum'}).reset_index()
grouped_ar7_df = filtered_ar7_df.groupby(['shortcode_DP.SV', 'Ar Day']).agg({'Revenue': 'sum', 'Cost': 'sum', 'Delivered': 'sum', 'Jump Page Clicks': 'sum', 'Clicks': 'sum', 'Opportunity Cost': 'sum', 'Optout': 'sum'}).reset_index()

In [147]:
#30 day averages
grouped_ar30_df['30 Day eCPM'] = grouped_ar30_df['Revenue'] / grouped_ar30_df['Delivered'] * 1000
grouped_ar30_df['30 Day CTR'] = grouped_ar30_df['Clicks'] / grouped_ar30_df['Delivered']
grouped_ar30_df['30 Day JCTR'] = grouped_ar30_df['Jump Page Clicks'] / grouped_ar30_df['Delivered']
grouped_ar30_df['30 Day Optout Rate'] = grouped_ar30_df['Optout'] / grouped_ar30_df['Delivered']

#7 day averages
grouped_ar7_df['7 Day eCPM'] = grouped_ar7_df['Revenue'] / grouped_ar7_df['Delivered'] * 1000
grouped_ar7_df['7 Day CTR'] = grouped_ar7_df['Clicks'] / grouped_ar7_df['Delivered']
grouped_ar7_df['7 Day JCTR'] = grouped_ar7_df['Jump Page Clicks'] / grouped_ar7_df['Delivered']
grouped_ar7_df['7 Day Optout Rate'] = grouped_ar7_df['Optout'] / grouped_ar7_df['Delivered']

In [148]:
# #percentages
grouped_ar30_df['30 Day CTR'] = grouped_ar30_df['30 Day CTR'] * 100
grouped_ar30_df['30 Day JCTR'] = grouped_ar30_df['30 Day JCTR'] * 100
grouped_ar30_df['30 Day Optout Rate'] = grouped_ar30_df['30 Day Optout Rate'] * 100

grouped_ar7_df['7 Day CTR'] = grouped_ar7_df['7 Day CTR'] * 100
grouped_ar7_df['7 Day JCTR'] = grouped_ar7_df['7 Day JCTR'] * 100
grouped_ar7_df['7 Day Optout Rate'] = grouped_ar7_df['7 Day Optout Rate'] * 100 

In [149]:
merged_df = grouped_ar30_df.merge(grouped_ar7_df[['shortcode_DP.SV','Ar Day', '7 Day eCPM']], 
                                 on=['shortcode_DP.SV', 'Ar Day'], 
                                 how='left')

In [150]:
# calculate percent difference between 30 day and 7 day eCPM with 30 day eCPM as the base
merged_df['eCPM % Diff'] = (merged_df['7 Day eCPM'] - merged_df['30 Day eCPM']) / merged_df['30 Day eCPM'] * 100

Most recent Offer ID for AR and Active

In [151]:
#Get Active SMS ARs from sms_ar
active_sms_ar = sms_AR[sms_AR['Status'] == 'Active']
active_sms_ar['shortcode_DP.SV'] = active_sms_ar['Shortcode'] + '_' + active_sms_ar['DP.DS']
#remove AR portion from Day column and convert the remaining number to an integer
active_sms_ar['Day'] = active_sms_ar['Day'].str.replace('AR', '').astype(float)
active_sms_ar.rename(columns={'Day': 'Ar Day'}, inplace=True)


<ipython-input-151-ce9b7e1a9952>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_sms_ar['shortcode_DP.SV'] = active_sms_ar['Shortcode'] + '_' + active_sms_ar['DP.DS']
<ipython-input-151-ce9b7e1a9952>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_sms_ar['Day'] = active_sms_ar['Day'].str.replace('AR', '').astype(float)
/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



In [152]:
#get active ARs from merged df
merged_df = merged_df.merge(active_sms_ar[['shortcode_DP.SV', 'Ar Day']], on=['shortcode_DP.SV', 'Ar Day'], how='inner')

In [153]:
merged_df = merged_df.merge(active_sms_ar[['shortcode_DP.SV', 'Ar Day', 'Offer ID', 'Date Started']], on=['shortcode_DP.SV', 'Ar Day'], how='left')
#convert Date Started to datetime
merged_df['Date Started'] = pd.to_datetime(merged_df['Date Started'])

#days since swap
merged_df['Days Since Swap'] = (datetime.now() - timedelta(days=1) - merged_df['Date Started']).dt.days

In [154]:
cols = list(merged_df.columns)
cols.insert(2, cols.pop(cols.index('Offer ID')))
merged_df = merged_df[cols]

In [155]:
#create a new column called AR Check, if eCPM % Diff is less than -25 and Days Since Swap is greater than 7, then AR Check is 'Underperforming', else if ecpm % diff is less than -25 and days since swap is less than 7, then AR Check is 'In Review', else 'Good'
merged_df['AR Check'] = merged_df.apply(lambda row: 'Downtrending - Swap' if row['eCPM % Diff'] < -25 and row['Days Since Swap'] > 37 
                                        else ('In Review' if row['eCPM % Diff'] < -25 and row['Days Since Swap'] <= 7
                                        else ('Underperforming Swap' if row['eCPM % Diff'] < -25 and row['Days Since Swap'] > 7 and row['Days Since Swap'] <= 37 else 'Good')), 
                                        axis=1)

In [156]:
#convert merged_df Offer ID to string
# merged_df['Offer ID'] = merged_df['Offer ID'].astype(str).str.replace(r'\.0$', '')

In [157]:
# BY DMA Account
merged_df['extracted_DP.SV'] = merged_df['shortcode_DP.SV'].str.split('_').str[-1]

#merge using 'extracted_DP.SV' and 'DP.SV'
merged_df = pd.merge(merged_df, pub_config[['DP.SV', 'DMA']], 
                     left_on='extracted_DP.SV', right_on='DP.SV', how='left')

#drop temp columns
merged_df = merged_df.drop(columns=['extracted_DP.SV', 'DP.SV'])

dma_separated_dfs = {}
for dma in merged_df['DMA'].unique():
    if pd.notna(dma):  # Ensure that the 'DMA' value is not NaN
        dma_separated_dfs[dma] = merged_df[merged_df['DMA'] == dma]

In [158]:
#new dataframes for statuses sorted
downtrending_swaps = merged_df[merged_df['AR Check'] == 'Downtrending - Swap']
downtrending_swaps = downtrending_swaps.sort_values(by=['DMA', 'shortcode_DP.SV','Days Since Swap'], ascending=[True, True, False])

in_review = merged_df[merged_df['AR Check'] == 'In Review']
in_review = in_review.sort_values(by=['DMA','shortcode_DP.SV', 'Days Since Swap'], ascending=[True, True, False])

underperforming_swaps = merged_df[merged_df['AR Check'] == 'Underperforming Swap']
underperforming_swaps = underperforming_swaps.sort_values(by=['DMA','shortcode_DP.SV', 'Days Since Swap'], ascending=[True, True, False])


In [159]:
#output file
filename = filepath.output_folder+'/AR Offer Alert - {}.xlsx'.format(datetime.today().strftime("%m_%d_%Y"))

#create writer
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
with pd.ExcelWriter(filename) as writer:
    for dma, df in dma_separated_dfs.items():
        sheet_name = f'{dma} - AR Offers'  # dynamic sheet name based on the DMA
        df.to_excel(writer, sheet_name=sheet_name, index=False)  # Writing data to different sheets

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    downtrending_swaps.to_excel(writer, sheet_name='Downtrending Swaps', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    underperforming_swaps.to_excel(writer, sheet_name='Underperforming Swaps', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    in_review.to_excel(writer, sheet_name='In Review', index=False)

In [160]:
email_body = "Hi Team, here is the Daily AR Offer Alert Report \n\n"
email_body += "Current Downtrending Threshold: 25%\n"
email_body += "Downtrending Swaps: eCPM % Diff < -25% and Days Since Last Swap > 37\n"
email_body += "Underperforming Swaps: eCPM % Diff < -25% and Days Since Last Swap > 7 and <= 37\n"
email_body += "In Review: eCPM % Diff < -25% and Days Since Last Swap <= 7\n\n"


email_body += "Please check the current downtrending ARs and swap:\n\n"
#add to email_body and for each DMA, add the count of downtrending swaps 
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {downtrending_swaps[downtrending_swaps['DMA'] == dma].shape[0]} downtrending swaps\n"

email_body += "\nPlease check the current underperforming swaps and swap back or to a new offer:\n\n"
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {underperforming_swaps[underperforming_swaps['DMA'] == dma].shape[0]} underperforming swaps\n"

email_body += "\nPlease check the recently swapped ARs that are in review and pay attention to performance changes:\n\n"
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {in_review[in_review['DMA'] == dma].shape[0]} swaps in review\n"

In [164]:
#show me merged_df where shortcode_DP.SV is SVT_AL.PL.2
merged_df[merged_df['shortcode_DP.SV'] == 'SVT_AL.PL.2']

,shortcode_DP.SV,Ar Day,Offer ID,Revenue,Cost,Delivered,Jump Page Clicks,Clicks,Opportunity Cost,Optout,30 Day eCPM,30 Day CTR,30 Day JCTR,30 Day Optout Rate,7 Day eCPM,eCPM % Diff,Date Started,Days Since Swap,AR Check,DMA
113,SVT_AL.PL.2,0.0,12113,3312.86,242.32320,48080.0,3700.0,6762.0,2335.890954,1933.0,68.903078,14.064060,7.695507,4.020383,92.463214,34.193154,2024-03-20,85.0,Good,Nathan Mai
114,SVT_AL.PL.2,1.0,13194,514.16,198.42480,39370.0,686.0,2336.0,-285.847570,1103.0,13.059690,5.933452,1.742443,2.801626,5.749295,-55.976790,2024-06-14,-1.0,In Review,Nathan Mai
115,SVT_AL.PL.2,2.0,11646,75.00,177.61968,35242.0,152.0,1102.0,-638.844006,663.0,2.128143,3.126951,0.431304,1.881278,2.218672,4.253934,2023-10-04,253.0,Good,Nathan Mai


In [161]:
toaddr = ['sms@rxmg.com','offernotices@rxmg.com']
today = datetime.today().strftime("%m_%d_%Y")
subject_line = f"SMS Daily AR Offer Alert Report - {today}"

for i in toaddr:
    send_email.send_email([filename],subject_line,email_body,i)